In [32]:
#trying to implement ensemble method
#https://datascience.stackexchange.com/questions/27169/taking-average-of-multiple-neural-networks
#mixture of experts 'with kmeans'
#https://en.wikipedia.org/wiki/Mixture_of_experts
#combining models together university of Tartu
#https://courses.cs.ut.ee/MTAT.03.277/2014_fall/uploads/Main/deep-learning-lecture-9-combining-multiple-neural-networks-to-improve-generalization-andres-viikmaa.pdf
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import keras
from keras.utils import to_categorical

(trX, trY), (teX, teY) = tf.keras.datasets.fashion_mnist.load_data()
trX = trX.reshape(60000, 784)
teX = teX.reshape(10000, 784)

trY = to_categorical(trY)
teY = to_categorical(teY)


print("x_train shape:", trX.shape, "y_train shape:", trY.shape)
print("x_test shape:", teX.shape, "y_test shape:", teY.shape)

x_train shape: (60000, 784) y_train shape: (60000, 10)
x_test shape: (10000, 784) y_test shape: (10000, 10)


In [39]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w_h1, w_h2, w_o):
    h1 = tf.nn.sigmoid(tf.matmul(X, w_h1)) # this is a basic mlp, think 2 stacked logistic regressions
    h = tf.nn.sigmoid(tf.matmul(h1, w_h2))
    #return tf.matmul(h, w_o, name="insertname_here") if we need to use names and save the models
    return tf.matmul(h, w_o) # note that we dont take the softmax at the end because our cost fn does that for us

#taken from https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def myRange(start,end,step):
    i = start
    while i < end:
        yield i
        i += step
    yield end

0
20
40
60
80
99


In [34]:
cluster_centers = np.load('kmeansclusters/' + str(models_to_train) + '.npy')
cluster_labels = np.load('kmeanslabels/' + str(models_to_train) + '.npy')

partitioned_train_data = []
partitioned_test_data = []
partitioned_train_labels = []
partitioned_test_labels = []

for i in range(0,models_to_train):
    partitioned_train_data.append([])
    partitioned_test_data.append([])
    partitioned_train_labels.append([])
    partitioned_test_labels.append([])

In [35]:
for i in range(0,models_to_train):
    for j in range(0,len(trX)):
        if cluster_labels[j] == i:
            partitioned_train_data[i].append(trX[j])
            partitioned_train_labels[i].append(trY[j])
    for j in range(0,len(teX)):
        if cluster_labels[j + len(trX)] == i:
            partitioned_test_data[i].append(teX[j])
            partitioned_test_labels[i].append(teY[j])

    partitioned_train_data[i] = np.vstack(partitioned_train_data[i])
    partitioned_test_data[i] = np.vstack(partitioned_test_data[i])
    partitioned_train_labels[i] = np.vstack(partitioned_train_labels[i])
    partitioned_test_labels[i] = np.vstack(partitioned_test_labels[i])
    
    print(partitioned_train_data[i].shape)
    print(partitioned_test_data[i].shape)
    print(partitioned_train_labels[i].shape)
    print(partitioned_test_labels[i].shape)

(17510, 784)
(2967, 784)
(17510, 10)
(2967, 10)
(13148, 784)
(2214, 784)
(13148, 10)
(2214, 10)
(9488, 784)
(1561, 784)
(9488, 10)
(1561, 10)
(10982, 784)
(1764, 784)
(10982, 10)
(1764, 10)
(8872, 784)
(1494, 784)
(8872, 10)
(1494, 10)


In [40]:
#saver = tf.train.Saver()

prediction = 0

model_accuracy = 0

models_to_train = 5

epochs_per_model = 10

batch_size = 128

# Launch the graph in a session
with tf.Session() as sess:
    for z in range(0,models_to_train):
        size_h1 = tf.constant(625, dtype=tf.int32)
        size_h2 = tf.constant(300, dtype=tf.int32)

        X = tf.placeholder("float", [None, 784])
        Y = tf.placeholder("float", [None, 10])

        w_h1 = init_weights([784, size_h1]) # create symbolic variables
        w_h2 = init_weights([size_h1, size_h2])
        w_o = init_weights([size_h2, 10])

        py_x = model(X, w_h1, w_h2, w_o)
        
        trX = partitioned_train_data[z]
        teX = partitioned_test_data[z]
        trY = partitioned_train_labels[z]
        teY = partitioned_test_labels[z]

        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)) # compute costs
        train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer
        predict_op = tf.argmax(py_x, 1)
        tf.global_variables_initializer().run()
        for i in range(epochs_per_model):
            for start, end in zip(myRange(0, len(trX), batch_size), myRange(batch_size, len(trX)+1, batch_size)):
                sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
            print(i, np.mean(np.argmax(teY, axis=1) ==
                             sess.run(predict_op, feed_dict={X: teX})))
        model_accuracy = np.sum(np.argmax(teY, axis=1) == sess.run(predict_op, feed_dict={X: teX})) + model_accuracy
        
    print("accuracy: " + str(model_accuracy/10000))
        
        #saver.save(sess,"mlp/session.ckpt")

0 0.627906976744186
1 0.7269969666329625
2 0.7401415571284126
3 0.7644084934277048
4 0.7694641051567239
5 0.7762049207954163
6 0.7893495112908662
7 0.8058645096056622
8 0.824401752612066
9 0.8267610380856084
0 0.3166214995483288
1 0.45076784101174344
2 0.546973803071364
3 0.564137308039747
4 0.509936766034327
5 0.6074977416440831
6 0.5298102981029811
7 0.45799457994579945
8 0.5998193315266486
9 0.5383920505871725
0 0.8327994875080077
1 0.840486867392697
2 0.8680333119795003
3 0.8923766816143498
4 0.8942985265855221
5 0.9051889814221653
6 0.9071108263933376
7 0.9141575912876362
8 0.9039077514413837
9 0.9026265214606022
0 0.8202947845804989
1 0.826530612244898
2 0.8361678004535147
3 0.8645124716553289
4 0.8673469387755102
5 0.8650793650793651
6 0.8673469387755102
7 0.8696145124716553
8 0.8679138321995464
9 0.8707482993197279
0 0.4457831325301205
1 0.5943775100401606
2 0.5970548862115127
3 0.607764390896921
4 0.6030789825970548
5 0.6318607764390897
6 0.6619812583668006
7 0.671352074966532